In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k, ndcg_at_k, ap_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

# Функция генерации фичей
from src.features import generate_fs

# Функция быстрого старта
from src.recommendation import tlvl_recommender

In [2]:
data = pd.read_csv('../raw_data/retail_train.csv')
item_features = pd.read_csv('../raw_data/product.csv')
user_features = pd.read_csv('../raw_data/hh_demographic.csv')

item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [3]:
recommender, pred = tlvl_recommender(data, week_sep=3)

NameError: name 'item_features' is not defined

In [ ]:
def get_recommendation(user, N=5):
    if user in data_train_lvl_1['user_id'].unique() and \
            user in data_train_lvl_2['user_id'].unique() and \
            user in data_val_lvl_2['user_id'].unique():
        return pred[pred['user_id'] == user, 'pred']
    elif (user in data_train_lvl_1['user_id'].unique() and \
            user in data_train_lvl_2['user_id'].unique()) or \
        (user in data_train_lvl_1['user_id'].unique() and \
            user in data_val_lvl_2['user_id'].unique()):
        return recommender.get_own_recommendations(user, N=N)
    else:
        return recommender.extend_with_top_popular([], N=N)

In [ ]:
get_recommendation(2375)